# Query

SELECT
 V.PDVID,
 V.SKUID,
 V.VENTA_NETA,
 V.KILOS,
 V.LITROS,
 V.FECHA,
 V.ANIO,
 V.SEMANA,
 V.DIA,
 P.TD_JEFATURA AS JEFATURA,
 P.CANAL_DEL_PDV AS CANAL,
 P.TD_NOMBRE AS DISTRIBUIDOR,
 COALESCE(SK1.CUP_SAP_CODE, SK2.CUP_SAP_CODE) AS CUP_SAP_CODE,
 COALESCE(SK1.CATEGORIA, SK2.CATEGORIA) AS CATEGORIA,
 COALESCE(SK1.SUBCATEGORIA, SK2.SUBCATEGORIA ) AS SUBCATEGORIA,
 COALESCE(SK1.SEGMENTO, SK2.SEGMENTO ) AS SEGMENTO
FROM
 `mdlz-orden-sugerida.raw_data.vw_ventas` V
INNER JOIN
 `mdlz-orden-sugerida.prep_data.pdvs_info` P
USING
 (PDVID)
LEFT JOIN
 `mdlz-orden-sugerida.prep_data.skus` SK1
USING
 (SKUID)
LEFT JOIN
 `mdlz-orden-sugerida.prep_data.skus` SK2
ON
 LPAD(CAST(V.SKUID AS STRING), 18, "0") = SK2.SKU_SAP_CODE
WHERE
 FECHA >= DATE_SUB(CURRENT_DATE(), INTERVAL 1 YEAR) -- Filtro de fecha
 AND P.TD_JEFATURA = 'ROSARIO' -- Filtro de jefatura
 AND COALESCE(SK1.CATEGORIA, SK2.CATEGORIA) IN ('Biscuits',
   'Chocolates',
   'Beverages',
   'Candies',
   'Gums',
   'Dry Mixes') -- Filtro de categoría
 AND P.CANAL_DEL_PDV NOT IN ('Otros',
   'Reventa') -- Filtro de canal
 AND COALESCE(SK1.CUP_SAP_CODE, SK2.CUP_SAP_CODE) IN (4007778, 658956, 655741, 660281, 660279, 666120, 4002809, 4027742, 548554, 548518, 4027744, 763027, 4015173, 548552, 548745, 323209, 323212, 548516, 32104, 786527, 4027626, 549271, 32258, 660487, 657662, 660283, 655755, 4015179, 4002648, 629792, 4027799, 4027739, 675928, 325190, 673233, 4027612, 32101, 548517, 4015197, 4027654, 920539, 655747, 4007784, 660201, 919451, 32058, 671126, 4015188, 4027655, 324473, 328008, 324477, 4002652, 4016383, 676351, 655763, 88847, 4027786, 4027740, 766115, 325193, 85948, 4015053, 4027656, 323216 )


# Setup and import data

In [1]:
import stan
import pandas as pd
import numpy as np
import scipy.stats as stats 
import matplotlib
import matplotlib.pyplot as plt
import arviz as az
from datetime import datetime
import multiprocessing as mp
import matplotlib.cm as cm
import time
#matplotlib.rcParams['figure.figsize']=5,5
matplotlib.rcParams['figure.dpi']=100
#plt.style.use('ggplot')

In [2]:
# this is a work around found at 
# https://stackoverflow.com/questions/56154176/runtimeerror-asyncio-run-cannot-be-called-from-a-running-event-loop-in-spyd
import nest_asyncio
nest_asyncio.apply()

In [3]:
#mycups = [4007778, 658956, 655741, 660281, 660279, 666120, 4002809, 4027742, 548554, 548518, 4027744, 763027, 4015173, 548552, 548745, 323209, 323212, 548516, 32104, 786527, 4027626, 549271, 32258, 660487, 657662, 660283, 655755, 4015179, 4002648, 629792, 4027799, 4027739, 675928, 325190, 673233, 4027612, 32101, 548517, 4015197, 4027654, 920539, 655747, 4007784, 660201, 919451, 32058, 671126, 4015188, 4027655, 324473, 328008, 324477, 4002652, 4016383, 676351, 655763, 88847, 4027786, 4027740, 766115, 325193, 85948, 4015053, 4027656, 323216, 628724, 326894, 324375, 4027646, 678229, 4027615, 4027593, 628720, 4027806, 660146, 971058, 4015200, 4027769, 32230, 753221, 629091, 4027644, 4027657, 4027714, 4027627, 4027666, 4020724, 655745, 4027578, 4015206, 676354, 12343, 326898, 631278, 4015191, 628726, 4027642, 628722, 659657, 969665, 628730, 4015203, 548955, 4001185, 4020728, 920623, 4027708, 979257, 4027807, 4014266, 4015185, 88861, 4027715, 4027805, 658959, 649401, 4027759, 4027658, 4016379, 666465, 4027779, 920631, 4027663, 631482, 655021, 376008, 757553, 4004756, 674813, 4027696, 9322200, 675123, 4018808, 4027598, 4027768, 649409, 4004776, 753219, 4027638, 662226, 13801, 4015182, 4027709, 677765, 32031, 4027698, 4027752, 4004760, 969300, 88271, 4027737, 4027695, 969673, 4027767, 4018835, 4027699, 678227, 4004762, 969267, 659666, 666122, 4004227, 4027662, 4027738, 675125, 675371, 660205, 969667, 32032, 655753, 4027755, 753217, 4027756, 4027753, 4027635, 4027780, 969229, 4027660, 4027754, 4027803, 4027762, 642911, 969694, 4027763, 4027797, 969297, 971851, 969691, 43518, 4027757, 4027804, 4027795, 763023, 4018850, 374530, 637141, 642627, 4027796, 4004406, 3320, 4019500, 969276, 4027746, 4015194, 4027724, 969306, 969656, 2669, 659655, 4027726, 4004409, 4019419, 4004415, 4027798, 854008, 4027800, 4019417, 4027777, 4016381, 854004, 4019421, 4004412, 971848, 678244, 4027801, 4019503, 969661, 666468, 249462, 4019497, 4027782, 324332, 673203]
topcups = pd.read_excel('data/vendimia/Top Productos.xlsx') 
topcups.sort_values(by='VENTA_NETA', ascending=False, inplace=True)
mycups = list(topcups.iloc[:65]['CUP_SAP_CODE']) # Select the top 65 
mycanals = ['Autoservicio', 'Kiosco Ventana', 'Maxi Kiosco', 'Tradicionales']
cups2names = pd.read_csv('data/vendimia/cups-to-names.csv')
mycups2 = []
for cc in mycups:
    dd = cups2names[cups2names['CUP_SAP_CODE']==cc]['CUPDESCRIPTION']
    mycups2.append([cc,[x for x in list(dd) if isinstance(x,str)]]) # This list contains first element the CUP, second element all the products represented by that CUP

In [4]:
my_model_parallel="""
functions{

real partial_sum(array[,] int sales_slice,
                   int start, int end,
                   vector d,
                   vector s,
                   int T,
                   int N)
{    
    real partial_target = 0;
    int slice_length = end-start+1;
    int t0;

   for (n in 1:slice_length) {
       t0 = 1;
       for (t in 1:T) {           
           if ( sales_slice[n,t] != 0 )
               {
               if ( t > 3 ) 
                   {                   
                   partial_target += poisson_lpmf ( sales_slice[n,t] | sum(d[t0:(t-1)]) * s[start+n-1] );
                   // error-in-our-way: CUP is not only one product, hence there is not such a thing as a unit, we should consider using a normal for real values here
                   }
               t0 = t;
               }        
     }; 
     };

    return partial_target;

}
}

data {
 int <lower=0> N; // number of PDVIDS
 int <lower=0> T; // number of days in the dataset
 int <lower=0> H; //Length of holidays list
 array[H] int<lower=0> holidays; // List of holidays numbered in absolut day format
 real <lower=0> my_max; // maximum number of sales per day in-some-average
 array[N,T] int<lower=0> sales;  // sales: is 0 if nothing sold, and x if that was sold in given daypractices[t,n] 
}

parameters {
  vector<lower=0, upper=1>[T-1] d; // parameter that measures how 'hot' is each day
  vector<lower=0, upper=my_max>[N] s; // parameter that estimates the size of each PDV
}

model {
  int grainsize = 1;
  for (h in 1:H)
    d[holidays[h]] ~ beta(1, 100); 
  target += reduce_sum(partial_sum, sales, grainsize, d, s, T, N);

}
"""

In [5]:
def compute_pdv(pdv_count):
    """ 
    We receive a PDV number (not ID) and use the inferred day-strength and size of this PDV to compute 
    the model prediction of each purchase, and compare it to the true purchase.  In this way we can
    assess how well the model works on this PDV for the CUP being used.

    The function returns a dataframe where for each purchase corresponds a line that compares the true to the 
    model prediction. Since there are many posterior samples, then the model prediction has a mean and a standard deviation
    therefore it also returns the significance = (true - predicted)/std.

    This a CPU consuming function, henceforth it is recommendable to parallelize it because of the so many PDVs
    """
    if pdv_count%50 == 0: print(pdv_count)
    structure = pd.DataFrame(columns=['PDVID', 'day', 'real_purchase', 'estimated_purchase', 'std_estimated_purchase', 'significance'])    
    # nzs has the indices of nonzeroes[1] where the pdv_count performed purchases, and each given element in nonzeroes[1]
    # has in which days the purchase was performed
    # (Observe: nonzeroes is a global variable, see the running code for its explanation)
    nzs = np.where(nonzeroes[0] == pdv_count)[0]
    last_day_non_zero = 0
    if np.random.uniform(0,1) < 0.2: 
        time.sleep(4) # this line and previous are to make it rest the cpus once in a while
    # nz numbers purchases of pdv_count
    for nz in nzs:    
        # The following line is a list that in each element it has the model prediction of the corresponding purchase
        # for each sample of the posterior.  Each df row is a sample of the posterior.
        # Observe that there ir a 'for' inside another 'for', henceforth it consumes CPU
        purchase_nz_predictions_for_each_posterior_sample = [sum(df.iloc[row][['d.'+str(i) for i in range(last_day_non_zero+1,nonzeroes[1][nz]+1)]]) * df.iloc[row]['s.'+str(pdv_count+1)] for row in range(len(df))]
        last_day_non_zero = nonzeroes[1][nz]
        mean = np.mean(purchase_nz_predictions_for_each_posterior_sample)
        std = np.std(purchase_nz_predictions_for_each_posterior_sample)
        structure.loc[nz] = [int(mydata2.iloc[pdv_count].name), nonzeroes[1][nz] + 1,         
                             inference_data[pdv_count][nonzeroes[1][nz]], 
                             mean, 
                             std, 
                             abs(inference_data[pdv_count][nonzeroes[1][nz]]-mean)/std] 
    return structure

In [6]:
#########################################
##                                     ##
##         Modify filename here        ##
##                                     ##
#########################################
filename = '20241204-rosario-full.csv' # Use here the file that comes out from the query in the first cell
dir = 'data/vendimia/'
mydata0 = pd.read_csv(dir+filename)
mydata0 = mydata0[(mydata0['ANIO'] == 2024) & (mydata0['KILOS'] > 0) & (mydata0['DISTRIBUIDOR'] == 'LA VENDIMIA S.RL.')]

In [7]:
#########################################
##                                     ##
##     Use cut below ONLY FOR TESTING  ##
##                                     ##
#########################################
#mydata0 = mydata0[mydata0['FECHA'] < '2024-10-26']

# Run

In [8]:
# disable chained assignments
pd.options.mode.chained_assignment = None 

In [9]:
%%time
details = []
counting = 0 
total = len(mycups) 
timestamp = datetime.now().strftime("%I%M%p%d%B%Y")
for mycup, name in mycups2[:]:
    #for mycanal in mycanals[:]:
    if 1:  # just to don't delete the indentation. 
           # It is better to do all canals simultaneously, the model realizes
           # that some PDVs are smaller and some other are larger through the variable 's[n]'
        mycanal = 'All'
        # Debugging info
        counting += 1
        print('+++++++++++++++++++++++++++++++++++ Running', counting, 'out of', total)
        
        # Define the mydata dataframe for this loop. That is, select only the purchases corresponding to the given 'mycup'
        # mydata = mydata0[(mydata0['CANAL']==mycanal) & (mydata0['CUP_SAP_CODE']==mycup)] # Un comment this and comment the next line if you want to run separately by canal
        mydata = mydata0[(mydata0['CUP_SAP_CODE']==mycup)] 
        mydata.loc[:,'day'] = (mydata['SEMANA']-1) * 7 + mydata['DIA']
        
        # Compute the weight of each unit of the given 'mycup'
        all_purchases_in_kilos=[]
        for i in range(len(mydata)):
            all_purchases_in_kilos.append(mydata.iloc[i]['KILOS'])
        all_purchases_in_kilos = list(set(all_purchases_in_kilos))
        all_purchases_in_kilos.sort()
        unit = all_purchases_in_kilos[0]

        # Let's get the frequency-of-purchase of each PDV´
        freq = []
        weeks = 1.0*max(mydata['day'])/7.0
        for pdvid in list(set(mydata['PDVID'])):
            pdvid_purchases = mydata[mydata['PDVID']==pdvid]
            #l = len(tmp)
            l = len(set(pdvid_purchases['day']))
            freq.append([pdvid, l, weeks/l ])        
        # create a second list of pdvid that purchase with a frequenccy of less than 7 weeks
        hfreq2 = [x[0] for x in freq if x[2] <7]

        # Select only those pdvid tha fulfill a frequency in purchases
        mydata1 = mydata[mydata['PDVID'].isin(hfreq2)]
        # Create a new column in mydata1 that has how many units was each purchase
        mydata1.loc[:,'units'] = mydata1['KILOS']/unit

        # Create and fill the dataframe with how many kilos were purchased each day by each PDVID
        # This dataframe is more basic that mydata1, but this is because we're preparing the array to be used in STAN
        mydata2 = pd.DataFrame(0, columns=range(1,max(mydata1['day'])+1), index=list(set(mydata1['PDVID'])))   # Add 1 because days count in naturals > 0        
        for i in range(len(mydata1['PDVID'])):
            mydata2.loc[ mydata1.iloc[i]['PDVID'] , mydata1.iloc[i]['day']] += int(mydata1.iloc[i]['units'])   

        # Find size of largest PDV in units-per-day
        tot = []
        for my_index in mydata2.index:
            tot.append(sum(mydata2.loc[my_index]))
        maxday = max(mydata1['day'])
        my_max = max(tot)/maxday # maximum per-day unit consumption in a given PDV

        # Convert mydata2 to an array to be used by STAN
        inference_data = np.array(mydata2.iloc[:, :]) # We convert all the available data into an array to inject the model with it
        inference_data_rows_pdvs = list(mydata2.index)

        # Create holidays vector. In these dates we set a prior of very few/none sales by each PDV. That is, the day strength is very likely to be close to zero
        holidays_date = [[1,1],[2,12],[2,13],[3,24],[3,29],[4,2],[5,1],[5,25],[6,17],[7,9],[8,19],[10,14],[11,20],[12,8],[12,25]]
        holidays = []
        for mydate in holidays_date:
            DAY_OF_YEAR = datetime(2024, mydate[0], mydate[1]).timetuple().tm_yday
            holidays.append(DAY_OF_YEAR)
        for week in range(1,52):
            holidays.append(week*7+0)        
        holidays = [d for d in holidays if d < inference_data.shape[1]]

        # Create dictionary that contains all the data with which STAN is fed
        stan_data = {'N': inference_data.shape[0], 'T': inference_data.shape[1], 'my_max': my_max, 'sales': inference_data, 'holidays': [int(i) for i in holidays], 'H': len(holidays)}

        # RUN STAN
        print('Running STAN with \nCUP:',mycup,'\nDescription:',name,'\nCanal:',mycanal )
        model = stan.build(my_model_parallel, data=stan_data, random_seed=0);
        fit = model.sample(num_chains=4, num_samples=1000 , num_warmup=1000, delta=0.8)
        df = fit.to_frame() 

        # Assessment
        # rhat is a variable that if below 1.05, means the inference and the posterior sampling is ok
        azdata = az.from_pystan(fit)
        summary = az.summary(azdata)
        rhat_mean = np.round(np.mean(summary['r_hat']), 3)
        rhat_std = np.round(np.std(summary['r_hat']), 3)

        #####
        ##### We have finished sampling, now write down the available info into myrow_d 

        # List of days strength in last 50 days
        # This is important to estimate sales in next days without the need of inferring the day-strength again
        # and therefore make it quick
        try:
            days_strengths=[]
            gap = 1
            last = 100
            dd = list(mydata1[mydata1['day']==maxday-last-gap]['DIA'])[0]
            # run i over the last 'last' days to get the mean day strength for each day
            for i in range(last+1):  # you have to add +1 because if you substract 'last' the you want to add 'last', but the range goes up to 'last'-1 if you don't add the +1
                day_strength_mean = np.mean(df['d.'+str(maxday-gap-last+i)])  
                # Following lines because some times there is no date
                day_fecha = mydata1[mydata1['day']==maxday-gap-last+i]['FECHA']
                if len(day_fecha) == 0:
                    fecha = 'no-in-data'
                else:
                    fecha = list(day_fecha)[0]
                days_strengths.append([fecha, (dd+i)%7, day_strength_mean])               
        except:
            days_strengths = [] 
        days_strengths = pd.DataFrame(days_strengths, columns=['date','weekday','day-strength'])

        # DF that contains the PDV info related to the current CUP
        pdvs_info_on_CUP = []
        for i in range(len(inference_data_rows_pdvs)):
            pdv = inference_data_rows_pdvs[i]
            units_purchased = [ x for x in mydata2.loc[pdv] if x > 0 ]
            i_th_pdv_size = df['s.'+str(i+1)]
            pdvs_info_on_CUP.append([pdv, np.mean(i_th_pdv_size), np.std(i_th_pdv_size), np.mean(units_purchased), np.std(units_purchased)])
        pdvs_info_on_CUP = pd.DataFrame(pdvs_info_on_CUP, columns = ['pdv', 'size_mean', 'size_std', 'units_purchased_mean', 'units_purchased_std'])

        # Perform assessment in parallel
        # Check how good is the agreement beyween past predictions and past true purchases
        # for each PDV, and use this results to assess how predictable is the given PDV for the given 'mycup'´
        nonzeroes = np.nonzero(inference_data)
        datapoints = np.count_nonzero(inference_data)
        parameters = len(list(df))
        print('Performing assessment:\ndatapoints = ',datapoints, 'parameters =', parameters)
        print('Total PDVs:', len(inference_data))
        pool = mp.Pool(25)
        results = pool.map(compute_pdv, [x for x in range(len(inference_data[:]))])
        pool.close()
        pool.join()        
        ## Collect all results from the parallelized run into one dataframe
        structure = pd.concat(results)

        ## Evaluate assessment
        structure2 = structure[structure['day']>20] # The first ~20 days we copnsider them transitory, and we do not epect predictions to be valid therein
        tt = np.corrcoef(structure2['real_purchase'],structure2['estimated_purchase'])
        R_pearson = np.round(tt[0][1],3)

        # Find error in each PDV prediction
        structure2.loc[:,'error'] = [abs(structure2.iloc[i]['real_purchase']-structure2.iloc[i]['estimated_purchase'])/structure2.iloc[i]['real_purchase'] for i in range(len(structure2))]
        metric = []
        for pdv_tmp in list(pdvs_info_on_CUP['pdv']):
            pdv_error = structure2[structure2['PDVID'] == pdv_tmp]['error']
            metric.append(np.mean(pdv_error))
        pdvs_info_on_CUP['error'] = metric
        
        ### Write down
        row = [mycup, name, mycanal, unit, [rhat_mean, rhat_std], [datapoints, parameters, datapoints/parameters], R_pearson, days_strengths, pdvs_info_on_CUP, maxday]
        details.append(row)
        details2 = pd.DataFrame(details, columns = ['CUP','Description','CANAL','unit/kg','Rhat','datapoints_and_parameters', 'R_pearson','days_strengths','unfolding-on-PDVs', 'maxday'])
        details2.to_pickle('data/vendimia/details_'+timestamp+'.pkl')
        time.sleep(10)
        

+++++++++++++++++++++++++++++++++++ Running 1 out of 65
Running STAN with 
CUP: 4007778 
Description: ['OREO PAQUETE                                      ', 'OREO PAQUETE                                      ', 'OREO REG C/CACAO 36X118G (76222018062700)', 'OREO REGULAR 36X118G (76222015219500)', 'OREO PAQUETE                                      ', 'OREO BANANA SPLIT 36X117G (76222011048100)', 'OREO REG 36X118G (76222015033100)', 'OREO REG M REL 42X181G PANDORA'] 
Canal: All


Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 53, column 12: The
    variable partial_sum may not have been assigned a value before its use.
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 52, column 29: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   5% (400/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:   9% (700/8000)
Sampling:  10% (800/8000)
Sampling:  11% (900/8000)
Sampli

Performing assessment:
datapoints =  19711 parameters = 1945
Total PDVs: 1604
0
350
250
400
100
300
200
50
650
450
150
600
500
700
750
800
550
850
900
950
1100
1000
1200
1150
1350
1050
1250
1400
1300
1500
1550
1450
1600
+++++++++++++++++++++++++++++++++++ Running 2 out of 65


Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 53, column 12: The
    variable partial_sum may not have been assigned a value before its use.
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 52, column 29: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.


Running STAN with 
CUP: 658956 
Description: ['PEPITOS PAQUETE                                   ', 'PEPITOS CHISPAS CADBURY 60X119G (76222018086500)', 'PEPITOS PAQUETE                                   ', 'PEPITOS REGULAR 60X119G (76222015032500)', 'PEPITOS PAQUETE                                   ', 'PEPITOS PAQUETE                                   '] 
Canal: All


...

Performing assessment:
datapoints =  555 parameters = 400
Total PDVs: 59
0
50


Building: found in cache, done.
Messages from stanc:
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 53, column 12: The
    variable partial_sum may not have been assigned a value before its use.
Warning in '/tmp/httpstan_23yh9fqh/model_fdrr3qp6.stan', line 52, column 29: Argument
    100 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
    provided, or the prior(s) depend on data variables. In the later case,
    this may be a false positive.
Sampling:   0%


+++++++++++++++++++++++++++++++++++ Running 65 out of 65
Running STAN with 
CUP: 323216 
Description: ['CLIGHT NARANJA DURAZNO 16X18X9', 'CLIGHT NARANJA DURAZNO 16X18X9', 'CLIGHT NAR DURA VITC+D 16X20X7,5G (76222018076400)', 'CLIGHT NAR DURAZNO GODZ 16X20X7.5G (76222014062600)', 'CLIGHT NARANJA DURAZNO 16X18X9'] 
Canal: All


Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)
Sampling:   1% (103/8000)
Sampling:   3% (202/8000)
Sampling:   4% (301/8000)
Sampling:   6% (500/8000)
Sampling:   8% (600/8000)
Sampling:  10% (800/8000)
Sampling:  12% (1000/8000)
Sampling:  14% (1100/8000)
Sampling:  16% (1300/8000)
Sampling:  18% (1400/8000)
Sampling:  19% (1500/8000)
Sampling:  20% (1600/8000)
Sampling:  22% (1800/8000)
Sampling:  25% (2000/8000)
Sampling:  28% (2200/8000)
Sampling:  29% (2300/8000)
Sampling:  30% (2400/8000)
Sampling:  32% (2600/8000)
Sampling:  34% (2700/8000)
Sampling:  36% (2900/8000)
Sampling:  39% (3100/8000)
Sampling:  54% (4300/8000)
Sampling:  70% (5600/8000)
Sampling:  86% (6900/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 8.1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.81 seconds.
  Adjust your expectations accordingly!


Performing assessment:
datapoints =  445 parameters = 389
Total PDVs: 48
0
CPU times: user 7min 28s, sys: 38.3 s, total: 8min 6s
Wall time: 2h 46min 25s
